In [ ]:
# SCRIPT TO FIND ALL SUBSIDIARIES, AS LISTED IN GOOGLE, OF ANY GIVEN COMPANY; AND THEN CREATE A REPORT BASED ON A QUERY OF THESE COMPANIES IN SCOUT, MAPEGYs INNOVATION GRAPH INTERFACE
# https://www.mapegy.com/en/

In [16]:
import os
import json 
import requests
import selenium
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
import base64
import time
import urllib.request

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from PIL import Image

import matplotlib.pyplot as plt
import cv2
from cv2 import dnn_superres

import re
import openai


In [40]:
# List of company names, added as continous string seperated by ","

companies = """NAMES OF COMPANIES""".split(", ")


In [18]:
# Path to chromedriver
DRIVER_PATH = 'chromedriver'
driver = webdriver.Chrome(DRIVER_PATH)

# Initiation to then manually accept or reject cookies
SUBSIDIARIES = f"https://www.google.com/search?q=initate"
driver.get(SUBSIDIARIES)

In [45]:

def google_subsidiaries(company):
    SUBSIDIARIES = f"https://www.google.com/search?q={company} subsidiaries"
    driver.get(SUBSIDIARIES)

    appbar = driver.find_element(By.CLASS_NAME, 'appbar')
    cards = appbar.find_elements(By.TAG_NAME, 'a')

    l = [company]
    for i in cards:
        l.append(i.text)

    l = list(filter(None, l))
    return l


company_lists = []
for company in companies:
    company_lists.append(google_subsidiaries(company))

In [47]:
char_remov = [',', '.', '&', '/', ' GmbH', 'limited', ' inc', ' Pty Ltd', ' Sp Z O O', ' LLC', ' company', ' corporation', ' mfg', ' sro', ' Ltd', ' a/s', ' srl']

edited_companies = []
for ind, i in enumerate(company_lists):
    direct_name = companies[ind].lower()
    l = [direct_name]
    for c in i:
        c = c.lower()
        c = c.replace('\n', ' ')
        for char in char_remov:
            char = char.lower()
            c = c.replace(char, " ")
            
        c = re.sub("[\(\[].*?[\)\]]", " ", c)
        c = re.sub("\s[a-z]{2}\s", " ", c)
        #c = re.sub("^[a-z]{2}\s", " ", c)
        #c = re.sub("\s[a-z]{2}$", " ", c)
        
        while True:
            s = c
            c = re.sub("\s[a-z]{1}\s", " ", c)
            c = re.sub("^[a-z]{1}\s", " ", c)
            c = re.sub("\s[a-z]{1}$", " ", c)
            c = re.sub("\s+", " ", c)
            if s == c:
                break
            
        c = c.strip()
        l.append(c)
        
    l = list(set(l))
    l = sorted(l, key=len)
    

    for tag in l:
        new_list = l.copy()
        new_list.remove(tag)
        for text in new_list:
            if re.search(tag, text):
                l.remove(text)
                
                
    edited_companies.append(l)

    
company_list = [(',').join(company) for company in edited_companies]


# SCOUT

In [26]:
openai.api_key = "OPEN AI API KEY"

pwd = 'MAPEGY SCOUT PWD'
user = 'MAPEGY SCOUT USERNAME'

template_folder = 'TEMPLATE FORDER NAME'
template_file = 'TEMPLATE FILE NAME'
save_folder = 'SAVE FOLDER NAME'


In [27]:
def gpt_api(query):
    chatgptPrompt = query
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": chatgptPrompt}
      ]
    ) 
    return completion.choices[0].message.content


In [28]:
driver.get('https://scout.mapegy.com/projects')

try:
    driver.find_element(By.ID, 'hs-eu-confirmation-button').click()

    navbar = driver.find_element(By.TAG_NAME, 'ul').find_elements(By.TAG_NAME, 'li')
    for i in navbar:
        if i.text.strip() == 'Login':
            i.click()
            break

    user_input = driver.find_element(By.ID, 'user_login')
    pwd_input = driver.find_element(By.ID, 'user_pass')

    user_input.send_keys(user)
    pwd_input.send_keys(pwd)

    driver.find_element(By.ID, 'wp-submit').click()
except:
    pass

In [49]:
import time

for i, company in enumerate(company_list):
    companies_ = companies
    query = company.replace(',', '%22,%22')
    driver.get(f"https://scout.mapegy.com/search?q=%7B%22format_version%22:%221.4.0%22,%22data%22:%7B%22topics%22:%5B%5B%5D%5D,%22not_topics%22:%5B%5D,%22organizations%22:%5B%5B%22{query}%22%5D%5D,%22not_organizations%22:%5B%5D,%22experts%22:%5B%5B%5D%5D,%22not_experts%22:%5B%5D%7D%7D")                                 


    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'filter-save-query')))
    time.sleep(1)
    driver.find_elements(By.CLASS_NAME, 'filter-save-query')[1].click()

    input_field = driver.find_element(By.TAG_NAME, 'input')
    driver.execute_script("arguments[0].value = ''", input_field)
    input_field.send_keys(companies_[i])                          

    # Template Selection

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)

    driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_elements(By.CSS_SELECTOR, "input[type='radio']")[1].click()  
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'mpg-confirm-dialog-btn'))).click()
    time.sleep(1)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)
    driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_element(By.TAG_NAME, 'input').send_keys(template_folder)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)
    lis = driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_elements(By.TAG_NAME, "li")

    for x in lis:
        if x.text.strip() == template_folder:
            x.find_element(By.TAG_NAME, "a").click()
            break
            
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)
    driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_element(By.TAG_NAME, 'input').send_keys(template_file)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)
    lis = driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_elements(By.TAG_NAME, "li")

    for x in lis:
        if x.text.strip() == template_file:
            x.find_element(By.TAG_NAME, "a").click()
            break
    
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'mpg-confirm-dialog-btn'))).click()
    time.sleep(1)    

    # Save Location

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)
    driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_element(By.TAG_NAME, 'input').send_keys(save_folder)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)
    lis = driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_elements(By.TAG_NAME, "li")

    for x in lis:
        if x.text.strip() == save_folder:
            x.find_element(By.TAG_NAME, "a").click()
            break
            
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)
    driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_elements(By.CSS_SELECTOR, "input[type='checkbox']")[-1].click()


    # Chat GPT Input & Saving

    gpt_query = f"What does the company {companies_[i]} do in 150 characters?"
    gpt_input = gpt_api(gpt_query)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'mpg-modal-content')))
    time.sleep(1)
    driver.find_element(By.CLASS_NAME, "mpg-modal-content").find_elements(By.CSS_SELECTOR, "input[type='text']")[-1].send_keys(gpt_input)  

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'mpg-confirm-dialog-btn'))).click()
    time.sleep(5)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.ID, 'mpg-confirm-dialog-btn'))).click()

    time.sleep(5)
    
    

In [ ]:
driver.quit()